In [1]:
import os
import glob
import torch
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn as nn
from torchnet import meter
from progressbar import * #进度条
from natsort import natsorted
import torch.nn.functional as F #torch是关于运算的包
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms as T
from torchvision import datasets,transforms, models #torchvision则是打包了一些数据集

#如果多gpu运行
os.environ['CUDA_VISIBLE_DEVICES']='2, 3' 
#否则
#os.environ['CUDA_VISIBLE_DEVICES']='2'

In [2]:
class BasicConv2d(nn.Module):
    def __init__(self, input_channels, output_channels, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(input_channels, output_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        
        return x

In [3]:
#same naive inception module
class InceptionA(nn.Module):
    def __init__(self, input_channels, pool_features):
        super().__init__()
        self.branch1x1 = BasicConv2d(input_channels, 64, kernel_size=1)
        
        self.branch5x5 = nn.Sequential(
            BasicConv2d(input_channels, 48, kernel_size=1),
            BasicConv2d(48, 64, kernel_size=5, padding=2)
        )
        
        self.branch3x3 = nn.Sequential(
            BasicConv2d(input_channels, 64, kernel_size=1),
            BasicConv2d(64, 96, kernel_size=3, padding=1),
            BasicConv2d(96, 96, kernel_size=3, padding=1),
        )
        
        self.branchpool = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(input_channels, pool_features, kernel_size=3, padding=1)
        )
    def forward(self, x):
        # x -> 1x1(same)
        branch1x1 = self.branch1x1(x)
        
        # x -> 1x1 -> 5x5(same)
        branch5x5 = self.branch5x5(x)
        
        # x -> 1x1 -> 3x3 -> 3x3(same)
        branch3x3 = self.branch3x3(x)
        
        # x -> pool -> 1x1(same)
        branchpool = self.branchpool(x)
        
        outputs = [branch1x1, branch5x5, branch3x3, branchpool]
        
        return torch.cat(outputs, 1)

In [4]:
#downsample
#Factorization into smaller convolutions
class InceptionB(nn.Module):
    def __init__(self, input_channels):
        super().__init__()
        self.branch3x3 = BasicConv2d(input_channels, 384, kernel_size=3, stride=2)
        
        self.branch3x3stack = nn.Sequential(
            BasicConv2d(input_channels, 64, kernel_size=1),
            BasicConv2d(64, 96, kernel_size=3, padding=1),
            BasicConv2d(96, 96, kernel_size=3, stride=2)
        )
        
        self.branchpool = nn.MaxPool2d(kernel_size=3, stride=2)
    def forward(self, x):
        # x -> 3x3(downsample)
        branch3x3 = self.branch3x3(x)
        
        # x -> 3x3 -> 3x3(downsample)
        branch3x3stack = self.branch3x3stack(x)
        
        # x -> avgpool(downsample)
        branchpool = self.branchpool(x)
        
        outputs = [branch3x3, branch3x3stack, branchpool]
        return torch.cat(outputs, 1)

In [5]:
#Factorizing Convolutions with Large Filter Size
class InceptionC(nn.Module):
    def __init__(self, input_channels, channels_7x7):
        super().__init__()
        self.branch1x1 = BasicConv2d(input_channels, 192, kernel_size=1)
        c7 = channels_7x7
        self.branch7x7 = nn.Sequential(
            BasicConv2d(input_channels, c7, kernel_size=1),
            BasicConv2d(c7, c7, kernel_size=(7, 1), padding=(3, 0)),
            BasicConv2d(c7, 192, kernel_size=(1, 7), padding=(0, 3))
        )
        
        self.branch7x7stack = nn.Sequential(
            BasicConv2d(input_channels, c7, kernel_size=1),
            BasicConv2d(c7, c7, kernel_size=(7, 1), padding=(3, 0)),
            BasicConv2d(c7, c7, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(c7, c7, kernel_size=(7, 1), padding=(3, 0)),
            BasicConv2d(c7, 192, kernel_size=(1, 7), padding=(0,3))
        )
        
        self.branch_pool = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(input_channels, 192, kernel_size=1)
        )
    def forward(self, x):
        # x -> 1x1(same)
        branch1x1 = self.branch1x1(x)
        # x -> 1layer 1*7 and 7*1 (same)
        branch7x7 = self.branch7x7(x)
        # x -> 2layer 1*7 and 7*1 (same)
        branch7x7stack =self.branch7x7stack(x)
        # x -> avgpool(same)
        branchpool = self.branch_pool(x)
        
        outputs = [branch1x1, branch7x7, branch7x7stack, branchpool]
        
        return torch.cat(outputs, 1)

In [6]:
class InceptionD(nn.Module):
    def __init__(self, input_channels):
        super().__init__()
        
        self.branch3x3 = nn.Sequential(
            BasicConv2d(input_channels, 192, kernel_size=1),
            BasicConv2d(192, 320, kernel_size=3, stride=2)
        )
        
        self.branch7x7 = nn.Sequential(
            BasicConv2d(input_channels, 192, kernel_size=1),
            BasicConv2d(192, 192, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(192, 192, kernel_size=(7, 1), padding=(3, 0)),
            BasicConv2d(192, 192, kernel_size=3, stride=2)
        )
        
        self.branchpool = nn.AvgPool2d(kernel_size=3, stride=2)
    def forward(self, x):
        # x -> 1x1 -> 3x3(downsameple)
        branch3x3 = self.branch3x3(x)
        # x -> 1x1 -> 1x7 -> 7x1 -> 3x3(downsample)
        branch7x7 = self.branch7x7(x)
        # x -> avgpool(downsampe)
        branchpool = self.branchpool(x)
        
        outputs = [branch3x3, branch7x7, branchpool]
        
        return torch.cat(outputs, 1)

In [7]:
#same
class InceptionE(nn.Module):
    def __init__(self, input_channels):
        super().__init__()
        self.branch1x1 = BasicConv2d(input_channels, 320, kernel_size=1)
        
        self.branch3x3_1 = BasicConv2d(input_channels, 384, kernel_size=1)
        self.branch3x3_2a = BasicConv2d(384, 384, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3_2b = BasicConv2d(384, 384, kernel_size=(3, 1), padding=(1, 0))
        
        self.branch3x3stack_1 = BasicConv2d(input_channels, 448, kernel_size=1)
        self.branch3x3stack_2 = BasicConv2d(448, 384, kernel_size=3, padding=1)
        self.branch3x3stack_3a = BasicConv2d(384, 384, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3stack_3b = BasicConv2d(384, 384, kernel_size=(3, 1), padding=(1, 0))
        
        self.branch_pool = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(input_channels, 192, kernel_size=1)
        )
    def forward(self, x):
        # x -> 1x1 (sample)
        branch1x1 = self.branch1x1(x)
        # x -> 1x1 -> 3x1
        # x -> 1x1 -> 1x3
        # concatenate(3x1, 1x3)
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = [
            self.branch3x3_2a(branch3x3),
            self.branch3x3_2b(branch3x3)
        ]
        branch3x3 = torch.cat(branch3x3, 1)
        # x -> 1x1 -> 3x3 -> 1x3
        # x -> 1x1 -> 3x3 -> 3x1
        # concatenate(1x3, 3x1)
        branch3x3stack = self.branch3x3stack_1(x)
        branch3x3stack = self.branch3x3stack_2(branch3x3stack)
        branch3x3stack = [
            self.branch3x3stack_3a(branch3x3stack),
            self.branch3x3stack_3b(branch3x3stack)
        ]
        
        branch3x3stack = torch.cat(branch3x3stack, 1)
        branchpool = self.branch_pool(x)
        outputs = [branch1x1, branch3x3, branch3x3stack, branchpool]
        return torch.cat(outputs, 1)

In [8]:
class InceptionV3(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.Conv2d_1a_3x3 = BasicConv2d(3, 32, kernel_size=3, padding=1) #(32-3+2)/1+1 = 32
        self.Conv2d_2a_3x3 = BasicConv2d(32, 32, kernel_size=3, padding=1)
        self.Conv2d_2b_3x3 = BasicConv2d(32, 64, kernel_size=3, padding=1)
        self.Conv2d_3b_1x1 = BasicConv2d(64, 80, kernel_size=1)
        self.Conv2d_4d_3x3 = BasicConv2d(80, 192, kernel_size=3) #(32-3+0)/1+1 = 30
        
        #naive inception module
        #3xInception
        self.Mixed_5b = InceptionA(192, pool_features=32)
        self.Mixed_5c = InceptionA(256, pool_features=64)
        self.Mixed_5d = InceptionA(288, pool_features=64)
        
        #downsample
        #5xInception
        self.Mixed_6a = InceptionB(288)
        
        self.Mixed_6b = InceptionC(768, channels_7x7=128)
        self.Mixed_6c = InceptionC(768, channels_7x7=160)
        self.Mixed_6d = InceptionC(768, channels_7x7=160)
        self.Mixed_6e = InceptionC(768, channels_7x7=192)
        
        #downsample
        #2xInception
        self.Mixed_7a = InceptionD(768)
        
        self.Mixed_7b = InceptionE(1280)
        self.Mixed_7c = InceptionE(2048)
        
        #6*6 feature size
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout2d()
        self.linear = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        # 32 -> 30
        x = self.Conv2d_1a_3x3(x)
        x = self.Conv2d_2a_3x3(x)
        x = self.Conv2d_2b_3x3(x)
        x = self.Conv2d_3b_1x1(x)
        x = self.Conv2d_4d_3x3(x)
        #30 -> 30
        x = self.Mixed_5b(x)
        x = self.Mixed_5c(x)
        x = self.Mixed_5d(x)
        #30 -> 14
        x = self.Mixed_6a(x)
        #14 -> 14
        x = self.Mixed_6b(x)
        x = self.Mixed_6c(x)
        x = self.Mixed_6d(x)
        x = self.Mixed_6e(x)
        #14 -> 6
        x = self.Mixed_7a(x)
        #6 -> 6
        x = self.Mixed_7b(x)
        x = self.Mixed_7c(x)
        #6 -> 1
        x = self.avgpool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [9]:
net = InceptionV3()
y = net(torch.randn(1, 3, 32, 32))
#print(net)
print(y.size())

torch.Size([1, 10])


In [10]:
label2int = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
int2lable = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [11]:
class TrainAndValData(Dataset):
    def __init__(self, img_path, csv_path, train=True, transforms=None):
        '''
        获得所有图片路径，并划分训练集、验证集
        '''
        self.train = train
        files = natsorted(glob.glob(img_path + '/*'))
        labels = pd.read_csv(csv_path).values[:, 1]
        files_num = len(files)
        break_point = int(0.9*files_num)
        if self.train:
            self.img_name = files[: break_point]
            self.img_label = labels[: break_point]
        else:
            self.img_name = files[break_point: ]
            self.img_label = labels[break_point: ]
         
   

        #数据增强
        if transforms is None:
            normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            #训练集用数据增强
            if self.train:
                self.transforms = T.Compose([
                    T.RandomCrop(32, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32`，
                    #T.Resize(256),
                    #T.RandomResizedCrop(224),
                    T.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
                    #T.RandomVerticalFlip(),
                    T.ToTensor(),
                    normalize 
                ])
            else:
                self.transforms = T.Compose([
                    T.Resize(32),
                    #T.CenterCrop(224),#中心裁剪
                    T.ToTensor(),
                    normalize 
                ])
    def __len__(self):
        '''
        返回数据集中所有图片的个数
        '''
        return len(self.img_name)
    def __getitem__(self, index):
        '''
        返回一张图片的数据
        '''
        img_path = self.img_name[index]
        img = Image.open(img_path)
        img = self.transforms(img)
        label = label2int[self.img_label[index]]
        return img, label
class TestData(Dataset):
    def __init__(self, img_path, transforms=None):
        files = natsorted(glob.glob(img_path + '/*'))
        self.img_name = files
        if transforms is None:
            normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            self.transforms =  T.Compose([
                T.Resize(32),
                T.ToTensor(),
                normalize
            ])
    def __len__(self):
        return len(self.img_name)
    def __getitem__(self, index):
        img_path = self.img_name[index]
        img = Image.open(img_path)
        img = self.transforms(img)
        return img

In [12]:
train_img_path = 'data/train'
csv_path = 'data/trainLabels.csv'
test_img_path = 'data/test'
train_dataset = TrainAndValData(train_img_path, csv_path, train=True)
val_dataset = TrainAndValData(train_img_path, csv_path, train=False)
test_dataset = TestData(test_img_path)

In [13]:
print('train len is %d' % len(train_dataset))
print(train_dataset[0][0].shape)
print(train_dataset[0][1])#打印标签
print('val len is %d' % len(val_dataset))
print(val_dataset[0][0].shape)
print(val_dataset[0][1])
print('test len is %d' % len(test_dataset))
print(test_dataset[0].shape)

train len is 45000
torch.Size([3, 32, 32])
6
val len is 5000
torch.Size([3, 32, 32])
7
test len is 300000
torch.Size([3, 32, 32])


In [14]:
# 超参数设置
EPOCH = 135   #遍历数据集次数
BATCH_SIZE = 128      #批处理尺寸(batch_size)
#LR = 0.001        #学习率
lr = 0.001
lr_decay = 0.99995
weight_decay = 1e-4
model_path = 'model/inception_v3/inception_v3.pkl'

In [15]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8) 

net = InceptionV3()

##单GPU
#net = net.cuda()
##多GPU
if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net, device_ids=[0,1])
#接着上一次训练
if os.path.exists(model_path):
    net = torch.load(model_path)
net = net.cuda()
# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
#optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
#保存模型判断条件
max_val_acc = 0
pre_epoch = 0
max_interval_epoch = 20
pre_train_loss = 100000

print("Start Training...")
for epoch in range(400):
    #训练集
    train_loss = 0
    train_count = 0
    net.train()
    for i, data in enumerate(trainloader):
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #更新指标
        train_count += labels.size(0)
        train_loss += loss.item()
    train_loss /= train_count

    #验证集
    val_acc = 0
    val_loss = 0
    val_count = 0
    net.eval()
    for i, data in enumerate(valloader):
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        inputs, labels =  inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        #更新指标
        val_count += 1
        val_loss += loss.item()
        _, predict = outputs.max(1)
        val_count += labels.size(0)
        val_acc += (predict == labels).sum().item()
    val_acc /= val_count
    val_loss /= val_count
    # print the loss and accuracy
    print('the epoch %d, the train loss is %f, the test loss is %f, the test acc is %f' % (epoch, train_loss, val_loss, val_acc))

    #保存模型
    if val_acc > max_val_acc:
        max_val_acc = val_acc
        pre_epoch = epoch
        torch.save(net, model_path)#保存整个神经网络的的结构信息和模型参数信息，save的对象是网络net
    if epoch - pre_epoch > max_interval_epoch:
        print('early stop')
        break

    #如果损失不载下降，则降低学习率
    if train_loss > pre_train_loss:
        lr = lr*lr_decay
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    pre_train_loss = pre_train_loss
print("Done Training!")

Start Training...
the epoch 0, the train loss is 0.010343, the test loss is 0.009208, the test acc is 0.570437


/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type InceptionV3. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type BasicConv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type InceptionA. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type InceptionB. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". 

the epoch 1, the train loss is 0.008625, the test loss is 0.007447, the test acc is 0.664484
the epoch 2, the train loss is 0.007405, the test loss is 0.006634, the test acc is 0.714286
the epoch 3, the train loss is 0.006308, the test loss is 0.006638, the test acc is 0.709524
the epoch 4, the train loss is 0.005585, the test loss is 0.005971, the test acc is 0.744048
the epoch 5, the train loss is 0.005014, the test loss is 0.004819, the test acc is 0.785516
the epoch 6, the train loss is 0.004581, the test loss is 0.004911, the test acc is 0.788690
the epoch 7, the train loss is 0.004102, the test loss is 0.005069, the test acc is 0.781548
the epoch 8, the train loss is 0.003820, the test loss is 0.004393, the test acc is 0.812302
the epoch 9, the train loss is 0.003510, the test loss is 0.004717, the test acc is 0.811310
the epoch 10, the train loss is 0.003242, the test loss is 0.004038, the test acc is 0.832143
the epoch 11, the train loss is 0.003016, the test loss is 0.004156, 